<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [8]:
import numpy as np
import cv2

In [9]:
img = cv2.imread("C:/Users/hp/Downloads/img_tested.jpg")

In [ ]:
cv2.imshow("scanit",img)
cv2.waitKey(0)

In [ ]:
img.shape

In [10]:
imgres = cv2.resize(img,(800,600))
cv2.imshow("scanit",imgres)
cv2.waitKey(0)

-1

In [11]:
img2=imgres.copy()


In [12]:

def event_check(event,x,y,flags,param):
    if(event==cv2.EVENT_RBUTTONDOWN):
        li.append((x,y))
li=[]
cv2.imshow("select",img2)
cv2.setMouseCallback("select",event_check)
if(cv2.waitKey(0)==27):
    cv2.destroyAllWindows()
        
        
       

In [13]:
print(li)
arr = np.array(li)

[(740, 61), (738, 497), (77, 510), (36, 49)]


In [14]:
from skimage.filters import threshold_local

In [15]:
def bird_view(image,pts):
    rec = np.zeros((4,2),dtype="float32")
    sumtotal = pts.sum(axis=1)
    rec[0]=pts[np.argmin(sumtotal)]
    rec[2]=pts[np.argmax(sumtotal)]
    difftotal=np.diff(pts,axis=1)
    rec[1]=pts[np.argmin(difftotal)]
    rec[3]=pts[np.argmax(difftotal)]
    w = max(int(np.sqrt((rec[0][0]-rec[1][0])**2+(rec[0][1]-rec[1][1])**2)),int(np.sqrt((rec[2][0]-rec[3][0])**2+(rec[2][1]-rec[3][1])**2)))
    h = max(int(np.sqrt((rec[0][0]-rec[3][0])**2+(rec[0][1]-rec[3][1])**2)),int(np.sqrt((rec[1][0]-rec[2][0])**2+(rec[1][1]-rec[2][1])**2)))
    final_pts = np.array([[0, 0],[w - 1, 0],[w - 1, h - 1],[0, h - 1]], dtype = "float32")
    transform_matrix = cv2.getPerspectiveTransform(rec,final_pts)
    img_birdview = cv2.warpPerspective(image,transform_matrix, (w, h))
    return(img_birdview)
birdview=bird_view(imgres,arr)
birdview = cv2.cvtColor(birdview, cv2.COLOR_BGR2GRAY)
birdview = cv2.fastNlMeansDenoising(birdview,None,10,7,21)
T = threshold_local(birdview, 13, offset = 10, method = "gaussian")
birdview = (birdview > T).astype("uint8") * 255
cv2.imshow("final",birdview)
cv2.waitKey(0)


-1

In [30]:
def rotate_in(image,angle):
   
    h=image.shape[0]
    w=image.shape[1]
    center = (h//2,w//2)
    imgrot=cv2.getRotationMatrix2D((center[1],center[0]),-angle,1.0)
    cos = np.abs(imgrot[0,0])
    sin =np.abs(imgrot[0,1])
    new_w = int((h*sin)+(w*cos))
    new_h = int((w*sin)+(h*cos))
    imgrot[0, 2] += (new_w / 2) - center[1]
    imgrot[1, 2] += (new_h / 2) - center[0]
    return(cv2.warpAffine(image,imgrot,(new_w,new_h)))
    
print("Enter the angle By which you would like to rotate : ")
angle = int(input())
cv2.imshow("Rotated",rotate_in(birdview,angle))
birdview = rotate_in(birdview,angle)
cv2.waitKey(0)


Enter the angle By which you would like to rotate : 
-90


-1

In [ ]:
print("Would you like to save the image ?")
print("Enter Yes or No")
n =input()
if(n.upper()=="YES"):
    print("Enter the file name : ")
    s = input()
    cv2.imwrite(s,birdview)

-1